## Drive Pathing:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch, torchvision
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import glob
import os
import ntpath
import numpy as np
import cv2
import random
import itertools
import pandas as pd
from tqdm import tqdm
import urllib
import json
import PIL.Image as Image
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.structures import BoxMode
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as pltfolds
from matplotlib import rc
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)


In [ ]:
!gdown --id 1K79wJgmPTWamqb04Op2GxW0SW9oxw8KSç

"gdown" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [ ]:
faces_df = pd.read_json('face_detection.json', lines=True)


In [ ]:
folds = {}
i_ = -1
# <major_axis_radius minor_axis_radius angle center_x center_y detection_score>.

PATH = folds__+'/'
for element in os.listdir(PATH):
    if 'ellipse' in element:
        i_ += 1
        folds[i_] = []
        with open(PATH + element,'r') as hdlr:
            lines = hdlr.readlines()
            for line in range(len(lines)):
                if '/' in lines[line]: 
                    new_image = [[],[]]
                    #print(lines[line]+ '.jpg')
                    pth = '/content/drive/MyDrive'
                    new_image[0] = cv2.imread(pth + '/'  + lines[line][:-1]+ '.jpg') 
                
                    for element in range(int(lines[line + 1])):
                        #print(lines[line + 2 + element][:-1].split(' '))
                        true = list(map(lambda x: float(x) , filter(lambda x: len(x) >0 ,lines[line + 2 + element][:-1].split(' '))))
                        xmin,xmax  = true[3] - true[1] , true[3] + true[1] 
                        ymin, ymax = true[4] - true[0] , true[4] + true[0]
                        width = true[0] *2
                        height = true[1] * 2
                        class_ = 'face'
                        file_name = lines[line][:-1]+ '.jpg'
                        data = {}
                        data['file_name'] = pth + '/' + file_name
                        data['width'] = height
                        data['height'] = width
                        data["x_min"] = xmin   
                        data["y_min"] = ymin
                        data["x_max"] = xmax
                        data["y_max"] = ymax
                        data['class_name'] = 'face'
                        new_image[1].append(data)
                    folds[i_].append(new_image)
        #    if new_image[0] == None: continue

In [ ]:
folds

{0: [[None,
   [{'class_name': 'face',
     'file_name': '2002/07/26/big/img_1010.jpg',
     'height': 130.26538,
     'width': 87.772244,
     'x_max': 86.501214,
     'x_min': -1.2710300000000032,
     'y_max': 212.076591,
     'y_min': 81.81121100000001},
    {'class_name': 'face',
     'file_name': '2002/07/26/big/img_1010.jpg',
     'height': 61.908372,
     'width': 44.279222,
     'x_max': 129.107864,
     'x_min': 84.828642,
     'y_max': 97.16984,
     'y_min': 35.261468},
    {'class_name': 'face',
     'file_name': '2002/07/26/big/img_1010.jpg',
     'height': 77.643462,
     'width': 50.802226,
     'x_max': 199.158218,
     'x_min': 148.355992,
     'y_max': 117.807949,
     'y_min': 40.164486999999994},
    {'class_name': 'face',
     'file_name': '2002/07/26/big/img_1010.jpg',
     'height': 167.870816,
     'width': 110.585152,
     'x_max': 383.312525,
     'x_min': 272.727373,
     'y_max': 255.221449,
     'y_min': 87.35063300000002},
    {'class_name': 'face',
     

In [ ]:
VAL_FOLD = 0
dataset = ([folds[x] for x in folds if x != VAL_FOLD])

In [ ]:
flatt = []
for x in dataset:
    flatt += x

In [ ]:
len(flatt)

2571

In [ ]:
def create_dataset_dicts(dataset):
  dataset_dicts = []
  for image_id, img in enumerate(dataset):
    record = {}
    image_df = img[0]
    file_path = img[1][0]['file_name']
    record["file_name"] = file_path
    record["image_id"] = image_id
    record["height"] = img[0].shape[0]
    record["width"] = img[0].shape[1]
    objs = []
    for row in img[1]:
      xmin = int(row['x_min'])
      ymin = int(row['y_min'])
      xmax = int(row['x_max'])
      ymax = int(row['y_max'])
      poly = [
          (xmin, ymin), (xmax, ymin),
          (xmax, ymax), (xmin, ymax)
      ]
      poly = list(itertools.chain.from_iterable(poly))
      obj = {
        "bbox": [xmin, ymin, xmax, ymax],
        "bbox_mode": BoxMode.XYXY_ABS,
        "segmentation": [poly],
        "category_id": 0,
        "iscrowd": 0
      }
      objs.append(obj)
    record["annotations"] = objs
    dataset_dicts.append(record)
  return dataset_dicts

classes = ['face']
for d in ["train", "val"]:
  DatasetCatalog.register("faces_" + d, lambda d=d: create_dataset_dicts(flatt if d == "train" else folds[VAL_FOLD]))
  MetadataCatalog.get("faces_" + d).set(thing_classes=classes)
statement_metadata = MetadataCatalog.get("faces_train")

In [ ]:
class CocoTrainer(DefaultTrainer):
  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):
    if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"
    return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(
  model_zoo.get_config_file(
    "COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"
  )
)
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(
  "COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"
)

cfg.DATASETS.TRAIN = ("faces_train",)
cfg.DATASETS.TEST = ("faces_val",)
cfg.DATALOADER.NUM_WORKERS = 8
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.WARMUP_ITERS = 1000
cfg.SOLVER.MAX_ITER = 2000
cfg.SOLVER.STEPS = (1000, 1500)
cfg.SOLVER.GAMMA = 0.05
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(classes)
cfg.TEST.EVAL_PERIOD = 500
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[12/20 15:26:57 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
model_final_2d9806.pkl: 431MB [00:38, 11.3MB/s]                           
Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.b

[12/20 15:27:40 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[12/20 15:28:53 d2.utils.events]:  eta: 1:56:48  iter: 19  total_loss: 1.912  loss_cls: 0.5493  loss_box_reg: 0.5414  loss_mask: 0.6884  loss_rpn_cls: 0.111  loss_rpn_loc: 0.01598  time: 3.6416  data_time: 0.0479  lr: 1.931e-06  max_mem: 6271M
[12/20 15:30:22 d2.utils.events]:  eta: 2:02:43  iter: 39  total_loss: 1.966  loss_cls: 0.56  loss_box_reg: 0.6616  loss_mask: 0.6808  loss_rpn_cls: 0.07606  loss_rpn_loc: 0.01706  time: 4.0672  data_time: 0.0072  lr: 2.911e-06  max_mem: 6271M
[12/20 15:31:44 d2.utils.events]:  eta: 2:01:28  iter: 59  total_loss: 1.996  loss_cls: 0.5338  loss_box_reg: 0.6049  loss_mask: 0.6684  loss_rpn_cls: 0.1098  loss_rpn_loc: 0.02009  time: 4.0721  data_time: 0.0089  lr: 3.891e-06  max_mem: 6271M
[12/20 15:33:06 d2.utils.events]:  eta: 2:00:23  iter: 79  total_loss: 1.766  loss_cls: 0.4929  loss_box_reg: 0.5746  loss_mask: 0.6482  loss_rpn_cls: 0.07887  loss_rpn_loc: 0.01335  time: 4.0857  data_time: 0.0084  lr: 4.871e-06  max_mem: 6271M
[12/20 15:34:36 d2.ut

In [ ]:
torch.save(trainer.model.state_dict(), os.path.join(cfg.OUTPUT_DIR, "mymodel.pth"))

In [ ]:
!ps -aux

"ps" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
